In [1]:
import pandas as pd
import numpy as np

In [2]:
import itertools
l_permutation = list(itertools.product([1, 2],[1,2],[1, 2],[1,2],[1, 2],[1,2],[1, 2],[1,2],[1, 2]))
df_permutation = pd.DataFrame(l_permutation,columns = ['beta','mu','p','delta','alpha','g_L','g_Z','g_Q','N_bar'])

In [3]:
class par: None
class moms: None
%run ./main.ipynb

Stored 'start' (str)
Stored 'end' (str)
Stored 'df_estimates' (DataFrame)
Stored 'start' (str)
Stored 'end' (str)
Stored 'df_estimates' (DataFrame)


In [7]:
#Assumptions and settings
par.sigma = 0.5
par.theta = 12
par.b = -np.log(1-0.15)
par.bh = -np.log((2-np.exp(-par.b)))

#Define equations
def growthpop(par,moms):
    return par.g_L

def priceinvt(par,moms):
    return -par.g_Q

def ik(par,moms):
    return (1+par.g_Q)*(1+par.g_T)-(1-par.delta)

def EmpPop(par,moms):
    return par.N_bar

def ProfitY(par,moms):
    return (par.mu+par.alpha-1)/(par.mu)

def TFPgrowth(par,moms):
    moms.ProfitY = (par.mu+par.alpha-1)/(par.mu)
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    return par.g_T-(1-moms.ProfitY)*par.g_L-moms.ProfitY*(par.g_T+par.g_Q)

def ProfitK(par,moms):
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma)*MOM
    r_star = 1/beta_star - 1
    return ((par.mu + par.alpha -1)/(par.alpha))*(r_star + par.delta + par.g_Q*(1+r_star))

def RF(par,moms):
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma) *MOM
    MOM2 = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))
    MOM3 = ((1-2*par.p)+par.p*np.exp(-par.bh*(-par.theta)) + par.p*np.exp(-par.b*(-par.theta))) 
    return MOM2/(MOM3*beta_star)-1

def PD(par,moms):
    par.b = -np.log(1-0.15)
    par.bh = -np.log((2-np.exp(-par.b)))
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta))     
    par.beta_star = par.beta * (1+par.g_PC)**(-par.sigma) *MOM
    #par.r_star = 1/beta_star - 1
    #par.beta_star = 1/(1+par.r_star)
    return par.beta_star*(1+par.g_T)/(1-par.beta_star*(1+par.g_T))

result = [(ProfitK(par,moms),ProfitY(par,moms),RF(par,moms), growthpop(par,moms), priceinvt(par,moms),PD(par,moms), TFPgrowth(par,moms), ik(par,moms), EmpPop(par,moms))
            for par.beta in [estimates.loc["beta","1984 - 2000"],estimates.loc["beta","2001 - 2016"]]
            for par.mu in [estimates.loc["mu","1984 - 2000"],estimates.loc["mu","2001 - 2016"]]
            for par.p in [estimates.loc["p","1984 - 2000"],estimates.loc["p","2001 - 2016"]]
            for par.delta in [estimates.loc["delta","1984 - 2000"],estimates.loc["delta","2001 - 2016"]]
            for par.alpha in [estimates.loc["alpha","1984 - 2000"],estimates.loc["alpha","2001 - 2016"]]
            for par.g_L in [estimates.loc["g_L","1984 - 2000"],estimates.loc["g_L","2001 - 2016"]]
            for par.g_Z in [estimates.loc["g_Z","1984 - 2000"],estimates.loc["g_Z","2001 - 2016"]]
            for par.g_Q in [estimates.loc["g_Q","1984 - 2000"],estimates.loc["g_Q","2001 - 2016"]]
            for par.N_bar in [estimates.loc["N_bar","1984 - 2000"],estimates.loc["N_bar","2001 - 2016"]]
         ]
df = pd.DataFrame(result, columns =["ProfitK","ProfitY","RF", "growthpop","priceinvt","PD","TFPgrowth","ik","EmpPop"])

In [8]:
df = df_permutation.join(df)

In [9]:
moments = ["ProfitK","ProfitY","RF","ik","TFPgrowth","priceinvt","PD","growthpop","EmpPop"]
parameters = ["beta","mu","p","delta","alpha","g_L","g_Z","g_Q","N_bar"]

df_results = pd.DataFrame(columns=parameters)
df_results["moment"] = moments
df_results = df_results.set_index("moment",drop=True)

for mom in moments:
    for parm in parameters:
        result = df.loc[(df[parm] == 2),mom].mean()-df.loc[(df[parm] == 1),mom].mean() 
        df_results.loc[mom,parm] = result    
df_results

,beta,mu,p,delta,alpha,g_L,g_Z,g_Q,N_bar
moment,,,,,,,,,
ProfitK,-0.0188303,0.0275645,0.0076221,0.00680054,4.3029e-05,8.32667e-17,-0.00292392,-0.0114947,0
ProfitY,0,0.0412945,0,0,-0.00025262,0,0,0,0
RF,-0.0121921,0,-0.0162462,0,-6.42094e-06,2.08167e-17,-0.00189317,-0.00103196,0
ik,0,0,0,0.00465221,-1.32853e-05,-0.000717553,-0.00391511,-0.00876273,0
TFPgrowth,0,-0.0014336,0,0,-1.39454e-07,-9.44378e-06,-0.00262662,0.000620515,0
priceinvt,0,0,0,0,0,0,0,0.00642559,0
PD,29.8002,0,-13.7868,0,-0.0189885,-2.02175,-5.4812,-3.00289,0
growthpop,0,0,0,0,0,-0.000693434,0,0,0
EmpPop,0,0,0,0,0,0,0,0,-0.0150664
